# Algoritmo Guloso

In [ ]:
import random
import time
import math
import matplotlib.pyplot as plt

class IGA:
    def __init__(self, dmin, dmax, y, z, orders):
        self.dmin = dmin
        self.dmax = dmax
        self.y = y
        self.z = z
        self.orders = orders
        self.best_schedule = None
        self.best_cost = float('inf')

    def initialize(self):
        total_production_times = [sum(order) for order in self.orders]
        sorted_orders = sorted(enumerate(total_production_times), key=lambda x: x[1])
        schedule = []

        for order_index, _ in sorted_orders:
            best_cost = float('inf')
            best_position = 0
            for position in range(len(schedule) + 1):
                temp_schedule = schedule[:position] + [order_index] + schedule[position:]
                cost = self.calculate_cost(temp_schedule)
                if cost < best_cost:
                    best_cost = cost
                    best_position = position

            schedule.insert(best_position, order_index)

        return schedule

    def calculate_cost(self, schedule):
        return sum(sum(self.orders[i]) for i in schedule)

    def local_search(self, schedule):
        for _ in range(int(self.z * self.dmin)):
            order_to_move = random.choice(schedule)
            schedule.remove(order_to_move)
            best_cost = float('inf')
            best_position = 0
            for position in range(len(schedule) + 1):
                temp_schedule = schedule[:position] + [order_to_move] + schedule[position:]
                cost = self.calculate_cost(temp_schedule)
                if cost < best_cost:
                    best_cost = cost
                    best_position = position

            schedule.insert(best_position, order_to_move)

        return schedule

    def destruction(self, schedule):
        orders_to_remove = random.sample(schedule, self.dmin)
        for order in orders_to_remove:
            schedule.remove(order)
        remaining_orders = [order for order in schedule]
        return orders_to_remove, remaining_orders

    def construction(self, removed_orders, remaining_orders):
        return self.nei_construction(remaining_orders, removed_orders)

    def nei_construction(self, remaining_orders, removed_orders):
        for order in removed_orders:
            best_cost = float('inf')
            best_position = 0
            for position in range(len(remaining_orders) + 1):
                temp_schedule = remaining_orders[:position] + [order] + remaining_orders[position:]
                cost = self.calculate_cost(temp_schedule)
                if cost < best_cost:
                    best_cost = cost
                    best_position = position

            remaining_orders.insert(best_position, order)

        return remaining_orders

    def accept(self, current_schedule, new_schedule):
        new_cost = self.calculate_cost(new_schedule)
        if new_cost < self.calculate_cost(current_schedule):
            return new_schedule
        else:
            q = random.uniform(0, 1)
            if q <= math.exp(-self.y * (self.dmin / self.dmax) * (new_cost - self.calculate_cost(current_schedule)) / self.calculate_cost(current_schedule)):
                return new_schedule
        return current_schedule

    def run(self, time_limit):
        current_schedule = self.initialize()
        start_time = time.time()
        
        while time.time() - start_time < time_limit:
            removed_orders, remaining_orders = self.destruction(current_schedule)
            new_schedule = self.construction(removed_orders, remaining_orders)
            current_schedule = self.local_search(new_schedule)
            current_schedule = self.accept(current_schedule, new_schedule)

        self.best_schedule = current_schedule
        self.best_cost = self.calculate_cost(current_schedule)

# Função para gerar instâncias de teste
def generate_instances(num_orders, num_machines):
    return [[random.randint(1, 100) for _ in range(num_machines)] for _ in range(num_orders)]

# Configuração experimental
time_limit = 10  # Em segundos
y = 1000
z = 2.0

# Parâmetros das instâncias de teste
test_instances = [
    (20, 5),
    (20, 10),
    (20, 20),
    (50, 5),
    (50, 10),
    (50, 20),
    (100, 5),
    (100, 10),
    (100, 20)
]

costs = []

# Executando o teste
for num_orders, num_machines in test_instances:
    dmin = max(1, num_orders // 2)
    dmax = 1
    orders = generate_instances(num_orders, num_machines)
    
    iga = IGA(dmin=dmin, dmax=dmax, y=y, z=z, orders=orders)
    iga.run(time_limit)
    
    costs.append((num_orders, num_machines, iga.best_cost))
    print(f"Pedidos: {num_orders}, Máquinas: {num_machines}")
    print(f"Melhor Cronograma: {iga.best_schedule}")
    print(f"Melhor Custo: {iga.best_cost}\n")

# Código para o gráfico de barras
labels = [f"{num_orders} pedidos, {num_machines} máquinas" for num_orders, num_machines, _ in costs]
values = [cost for _, _, cost in costs]

plt.figure(figsize=(12, 6))
plt.bar(labels, values, color='skyblue')
plt.xticks(rotation=45, ha='right')
plt.xlabel("Configurações de Instância")
plt.ylabel("Melhor Custo Total (TCT)")
plt.title("Comparação de Custos Totais para Diferentes Configurações de Pedidos e Máquinas")
plt.show()

Pedidos: 20, Máquinas: 5
Melhor Cronograma: [5, 12, 7, 1, 19, 10, 13, 11, 0, 14, 2, 3, 6, 17, 8, 18, 16, 9, 15, 4]
Melhor Custo: 4982

Pedidos: 20, Máquinas: 10
Melhor Cronograma: [10, 18, 14, 1, 19, 8, 6, 2, 7, 0, 5, 16, 11, 9, 4, 12, 13, 17, 3, 15]
Melhor Custo: 10208

Pedidos: 20, Máquinas: 20
Melhor Cronograma: [2, 1, 13, 4, 6, 10, 12, 18, 9, 0, 17, 7, 14, 11, 3, 5, 15, 19, 8, 16]
Melhor Custo: 20740

Pedidos: 50, Máquinas: 5
Melhor Cronograma: [8, 30, 6, 15, 45, 19, 9, 18, 31, 10, 47, 33, 42, 35, 49, 24, 28, 20, 21, 7, 11, 26, 48, 17, 25, 32, 39, 5, 46, 12, 1, 38, 22, 13, 34, 2, 4, 40, 29, 3, 14, 16, 23, 0, 27, 44, 43, 41, 37, 36]
Melhor Custo: 12847

Pedidos: 50, Máquinas: 10
Melhor Cronograma: [3, 21, 2, 24, 48, 42, 19, 0, 38, 5, 41, 25, 7, 16, 11, 32, 6, 12, 46, 43, 49, 37, 34, 15, 30, 22, 1, 47, 23, 17, 4, 18, 45, 8, 20, 28, 40, 33, 27, 10, 26, 31, 13, 29, 9, 44, 14, 35, 36, 39]
Melhor Custo: 25174

Pedidos: 50, Máquinas: 20
Melhor Cronograma: [28, 48, 21, 1, 24, 41, 4, 40, 14